In [9]:
import mlflow
import pandas as pd

In [1]:
from sklearn.metrics import f1_score, accuracy_score

def print_predictions(m, y_test, y_pred):
    # The coefficients
    print("Coefficients: \n", m.coef_)
    print("Mean squared error: %.2f" % f1_score(y_test, y_pred))
    print("accuracy_score: %.2f" % accuracy_score(y_test, y_pred))

# Inference for XGboost

In [4]:
from pathlib import Path
from sangkak_estimators import SangkakPosProjetReader, SangkakPosFeaturisation

language = "bbj"
augment = False

# Get path of test data 
pos_path  = Path(f'../data_source/masakhane-pos/data/{language}')
test_data_path = pos_path / 'test.txt'

# read data from source with sklearn estimator
reader_estimator = SangkakPosProjetReader()
list_test_data, _ = reader_estimator.fit(test_data_path).\
                                    transform_analysis(augment=augment)

feature_estimator = SangkakPosFeaturisation()
feature_estimator.fit([])

Xtest = feature_estimator.transform(list_test_data)
ytest  = feature_estimator.transform(list_test_data, label=True)

features_types = {
    "categorical_features": [
        'word.letters',
        'word',
        'word.tones',
        'word.normalized',
        'word.lower()',
        'word.prefix',
        'word.root',
        '-1:word',
        #'-1:word.tag',
        '-1:word.normalized',
        '-1:word.lower()',
        '-1:word.prefix',
        '-1:word.root',
        '-1:word.letters',
        '+1:word.letters',
        '+1:word.prefix',
        '+1:word.root',
        '+1:word',
        '+1:word.lower()',
        #'+1:word.tag',
        '+1:word.normalized'
    ],
    "numerical_features": [
        'bias',
        'word.position',
        'word.have_tone',
        'word.ispunctuation',
        'word.isdigit()',
        'word.EOS',
        'word.BOS',
        'word.start_with_capital',
        'word.has_hyphen',
        '-1:word.position',
        '-1:word.start_with_capital',
        '-1:len(word-1)',
        '+1:word.position',
        '+1:word.start_with_capital',
        '+1:len(word+1)',
        '+1:word.isdigit()',
        '+1:word.ispunctuation',
        '+1:word.BOS',
        '+1:word.EOS',
        '-1:word.isdigit()',
        '-1:word.ispunctuation',
        '-1:word.BOS',
        '-1:word.EOS'
    ]
}

xgb_df_test = feature_estimator.transform_to_sagemaker_format(
    Xtest, ytest, 
    label='test',
    normalize=features_types
)

-> Read input sentences
-> Featurisation of input sentences
-> Featurisation of input sentences
-> [test] Building sagemaker data for classification


In [12]:
labels = xgb_df_test["labels"]
features = xgb_df_test.drop("labels", axis=1)

features

,word,bias,word.tones,word.normalized,word.position,word.has_hyphen,word.lower(),word.start_with_capital,word.have_tone,word.prefix,...,+1:word.position,+1:word.letters,+1:word.normalized,+1:word.start_with_capital,+1:len(word+1),+1:word.lower(),+1:word.isdigit(),+1:word.ispunctuation,+1:word.BOS,+1:word.EOS
0,.,1,,.,15,0,.,0,0,,...,16,-1,,-1,-1,,-1,-1,0,1
1,áá,1,,áá,14,0,áá,0,1,á,...,15,.,.,0,1,.,0,1,0,1
2,Afríka,1,,Afríka,13,0,afríka,1,1,Af,...,14,a ́ a ́,áá,0,4,áá,0,0,0,0
3,shya,1,,shya,12,0,shya,0,0,sh,...,13,A f r i ́ k a,Afríka,1,7,afríka,0,0,0,0
4,pə̂,1,̂,pə̂,11,0,pə̂,0,1,pə,...,12,s h y a,shya,0,4,shya,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9285,mjwǐ,1,̌,mjwǐ,4,0,mjwǐ,0,1,mj,...,5,F o g u ŋ,Foguŋ,1,5,foguŋ,0,0,0,0
9286,yə,1,,yə,3,0,yə,0,0,,...,4,m j w i ̌,mjwǐ,0,5,mjwǐ,0,0,0,0
9287,mú,1,,mú,2,0,mú,0,1,mu,...,3,y ə,yə,0,2,yə,0,0,0,0
9288,Biya,1,,Biya,1,0,biya,1,0,Bi,...,2,m u ́,mú,0,3,mú,0,0,0,0


In [17]:

logged_model = 'runs:/2578ff6d907d48ed8c56a6cb771253af/xgboost-model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
predicted = loaded_model.predict(features.iloc[:4, :])


predicted

MlflowException: Run '2578ff6d907d48ed8c56a6cb771253af' not found

In [19]:
reg_model_name = "xgboost-classifier-model"
model_uri = f"models:/{reg_model_name}/8"

loaded_model = mlflow.xgboost.load_model(model_uri)
print("--")

# Make predictions using the testing set
diabetes_y_pred = loaded_model.predict(features.iloc[:4, :])
print(diabetes_y_pred)


MlflowException: Registered Model with name=xgboost-classifier-model not found

In [20]:
dir(mlflow)

['ActiveRun',
 'LazyLoader',
 'MlflowClient',
 'MlflowException',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_configure_mlflow_loggers',
 '_doctor',
 'active_run',
 'artifacts',
 'autolog',
 'azure',
 'catboost',
 'client',
 'contextlib',
 'create_experiment',
 'data',
 'delete_experiment',
 'delete_run',
 'delete_tag',
 'diviner',
 'doctor',
 'end_run',
 'entities',
 'environment_variables',
 'evaluate',
 'exceptions',
 'fastai',
 'get_artifact_uri',
 'get_experiment',
 'get_experiment_by_name',
 'get_parent_run',
 'get_registry_uri',
 'get_run',
 'get_tracking_uri',
 'gluon',
 'h2o',
 'is_tracking_uri_set',
 'johnsnowlabs',
 'langchain',
 'last_active_run',
 'lightgbm',
 'llm',
 'load_table',
 'log_artifact',
 'log_artifacts',
 'log_dict',
 'log_figure',
 'log_image',
 'log_input',
 'log_metric',
 'log_metrics',
 'log_param',
 'log_params',
 'log_table',
 'log_text',
 'ml